In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import scipy.stats.distributions as dist
import holoviews as hv
import hvplot.pandas
%matplotlib inline


In [ ]:
# Importing the data for training year and board certification year
# Setting the path of the csv file
data_file = Path("Decision_Analytics_TeacherDataset.xlsx")
# Importing the data file and setting the teacher id as the index column.
df = pd.read_excel(data_file)
df.set_index('teacher_id', inplace = True)
# Reviewing first 5 rows of the DataFrame
df.head()

In [ ]:
# Importing the data on the evaluation scores
# Setting the path of the csv file
data_file = Path("Decision_Analytics_TeacherDataset.xlsx")
# Importing the data file and setting the teacher id as the index column.
df2 = pd.read_excel(data_file, sheet_name='performance_evaluations',skiprows=1)
# Reviewing first 5 rows of the DataFrame
df2 = pd.melt(df2, id_vars='teacher_id',var_name='year',value_name='eval_score')
df2.set_index('teacher_id',inplace = True)
df2.head()

In [ ]:
df['pass'] = df['training_year'] == df['board_cert_year']
df['prior90'] = df['training_year'] < 1990
df = df.astype({'pass': int, 'prior90': int})
df.describe()

In [ ]:
df.groupby(['training_year']).agg({'pass': np.mean}).hvplot.line()

In [ ]:
#Drop anomaly years
indexYears = df[(df['training_year'] == 1991) | (df['training_year'] == 1993) | (df['training_year'] == 1995) | (df['training_year'] >= 2000)].index
df.drop(indexYears , inplace=True)
df.head(15)

In [ ]:
df.groupby(['training_year']).agg({'pass': np.mean}).hvplot.line()

In [ ]:
#Assemble frequencies and encode the variables
dftest=df.loc[:, ["pass","prior90"]]
dftest['pass'] = dftest['pass'].map({1:'Passed on 1st Try',0:'Failed on 1st Try'}) 
dftest['prior90'] = dftest['prior90'].map({1:'Before 1990',0:'1990 or Later'}) 

In [ ]:
contingency_table = pd.crosstab(dftest['pass'],dftest['prior90'])
contingency_table

In [ ]:
contingency_table = pd.crosstab(dftest['pass'],dftest['prior90']).apply(lambda r:r/r.sum(),axis=0)
contingency_table

In [ ]:
# Join the evaluation scores with certification data